In [ ]:
import pandas as pd
from datetime import datetime

def analyze_shipment_data(file_path):
    """
    Analyze shipment data from Excel file and generate metrics by District Name and Referral Month.
    This version searches for columns by their content pattern, not by column names.
    
    Args:
        file_path (str): Path to the Excel file
    
    Returns:
        pandas.DataFrame: Summary data with metrics
    """
    
    try:
        # Read the Excel file with multiple strategies
        print(f"Reading file: {file_path}")
        
        # Try reading with different parameters
        try:
            # First attempt - normal read
            df = pd.read_excel(file_path)
        except:
            try:
                # Second attempt - skip first row if it's merged cells or formatting
                df = pd.read_excel(file_path, header=1)
            except:
                # Third attempt - no header, we'll find it ourselves
                df = pd.read_excel(file_path, header=None)
        
        print(f"DataFrame shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        
        # If all columns are unnamed, try to find the header row
        if all('Unnamed' in str(col) for col in df.columns) or all(df.columns.isna()):
            print("Detecting header row...")
            # Look for the row that contains our target columns
            target_terms = ['district', 'referral', 'shipment']
            
            for i in range(min(10, len(df))):  # Check first 10 rows
                row_str = ' '.join(str(val).lower() for val in df.iloc[i] if pd.notna(val))
                if all(term in row_str for term in target_terms):
                    print(f"Found header row at index {i}")
                    # Use this row as column names
                    df.columns = df.iloc[i]
                    df = df.iloc[i+1:].reset_index(drop=True)
                    break
        
        # Now find the columns by pattern matching
        district_col = None
        referral_col = None
        shipment_col = None
        
        # Search for columns by content pattern, not name
        print("\nSearching for columns by content...")
        
        # Look through each column to find the right ones
        for col in df.columns:
            # Get sample values from this column (first 100 non-null values)
            sample_values = df[col].dropna().head(100).astype(str)
            
            if sample_values.empty:
                continue
                
            # Join sample values to search for patterns
            sample_text = ' '.join(sample_values.str.lower())
            
            # Check for district patterns
            if not district_col and 'district' in sample_text:
                district_col = col
                print(f"Found district column: {col}")
            
            # Check for referral month patterns (look for date formats)
            if not referral_col:
                # Look for patterns like YYYY-MM or mentions of months
                date_patterns = any(x in sample_text for x in ['2024', '2025', 'jan', 'feb', 'mar', 'apr', 'may', 'jun'])
                if date_patterns or 'referral' in str(col).lower():
                    # Verify it contains date-like values
                    date_count = sum(1 for val in sample_values if '-' in val and len(val) == 7)
                    if date_count > len(sample_values) * 0.1:  # At least 10% are date-like
                        referral_col = col
                        print(f"Found referral month column: {col}")
            
            # Check for shipment status patterns
            if not shipment_col:
                shipment_terms = ['ship', 'did not ship', 'less than 84', 'shipment']
                if any(term in sample_text for term in shipment_terms):
                    shipment_col = col
                    print(f"Found shipment status column: {col}")
        
        # If still not found, check column names more broadly
        if not all([district_col, referral_col, shipment_col]):
            print("\nTrying column name matching...")
            for col in df.columns:
                col_str = str(col).lower()
                if not district_col and 'district' in col_str:
                    district_col = col
                if not referral_col and ('referral' in col_str or 'month' in col_str):
                    referral_col = col
                if not shipment_col and ('shipment' in col_str or 'ship' in col_str):
                    shipment_col = col
        
        if not all([district_col, referral_col, shipment_col]):
            print("Error: Could not identify all required columns")
            print("Please check these column samples:")
            
            # Show first few rows of data to help identify columns
            print("\nFirst 5 rows of data:")
            print(df.head())
            
            # Show unique values in each column
            for col in df.columns:
                print(f"\nColumn '{col}' sample values:")
                print(df[col].dropna().head(10).tolist())
            
            return None
        
        print(f"\nUsing columns:")
        print(f"District: {district_col}")
        print(f"Referral Month: {referral_col}")
        print(f"Shipment Status: {shipment_col}")
        
        # Clean the data
        df[district_col] = df[district_col].astype(str).str.strip()
        df[referral_col] = df[referral_col].astype(str).str.strip()
        df[shipment_col] = df[shipment_col].astype(str).str.strip()
        
        # Remove rows with missing values in key columns
        df = df.dropna(subset=[district_col, referral_col, shipment_col])
        
        # Check what shipment statuses we have
        print(f"\nUnique shipment statuses found:")
        print(df[shipment_col].unique())
        
        # Filter for relevant shipment statuses (case insensitive)
        df_filtered = df[df[shipment_col].str.lower().isin(['ship in less than 84 days', 'did not ship'])].copy()
        
        if df_filtered.empty:
            print("Warning: No rows found with 'Ship in less than 84 days' or 'Did not ship' status")
            # Try fuzzy matching
            print("Trying fuzzy matching for shipment statuses...")
            statuses = df[shipment_col].unique()
            for status in statuses:
                if '84' in str(status) or 'ship' in str(status).lower():
                    print(f"Found potential match: {status}")
            return None
        
        # Group by District Name and Referral Month
        results = []
        
        grouped = df_filtered.groupby([district_col, referral_col])
        
        for (district, month), group in grouped:
            # Count occurrences of each status
            status_counts = group[shipment_col].value_counts()
            
            ship_less_84 = status_counts.get('Ship in less than 84 days', 0)
            did_not_ship = status_counts.get('Did not ship', 0)
            total_shipments = ship_less_84 + did_not_ship
            
            # Calculate percentage (avoid division by zero)
            if total_shipments > 0:
                ship_percentage = ship_less_84 / total_shipments
            else:
                ship_percentage = 0
            
            results.append({
                'District Name': district,
                'Referral Month': month,
                'Ship in less than 84 days': ship_less_84,
                'Did not ship': did_not_ship,
                'Total shipments': total_shipments,
                'Ship percentage': ship_percentage,
                'Ship percentage (%)': round(ship_percentage * 100, 2)
            })
        
        # Create results DataFrame
        results_df = pd.DataFrame(results)
        
        if not results_df.empty:
            # Calculate totals for each district across all months
            district_totals = []
            
            for district in results_df['District Name'].unique():
                district_data = results_df[results_df['District Name'] == district]
                
                total_ship_less_84 = district_data['Ship in less than 84 days'].sum()
                total_did_not_ship = district_data['Did not ship'].sum()
                total_shipments = total_ship_less_84 + total_did_not_ship
                
                if total_shipments > 0:
                    total_ship_percentage = total_ship_less_84 / total_shipments
                else:
                    total_ship_percentage = 0
                
                district_totals.append({
                    'District Name': district,
                    'Referral Month': 'ALL MONTHS - TOTAL',
                    'Ship in less than 84 days': total_ship_less_84,
                    'Did not ship': total_did_not_ship,
                    'Total shipments': total_shipments,
                    'Ship percentage': total_ship_percentage,
                    'Ship percentage (%)': round(total_ship_percentage * 100, 2)
                })
            
            # Create DataFrame for district totals
            district_totals_df = pd.DataFrame(district_totals)
            
            # Combine original results with district totals
            results_df = pd.concat([results_df, district_totals_df], ignore_index=True)
            
            # Sort: First by District Name, then put totals after individual months
            results_df['sort_key'] = results_df['Referral Month'].apply(
                lambda x: 'z_total' if 'ALL MONTHS' in str(x) else str(x)
            )
            results_df = results_df.sort_values(['District Name', 'sort_key'])
            results_df = results_df.drop('sort_key', axis=1)
        
        return results_df
    
    except Exception as e:
        print(f"Error processing file: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Run the analysis
file_path = "frminfo.xlsx"  # Change this if your file is in a different location

# Analyze the data
results = analyze_shipment_data(file_path)

if results is not None:
    print("\nAnalysis complete!")
    print(f"Total combinations found: {len(results)}")
    
    # Display results (Jupyter will show this nicely formatted)
    display(results)
    
    # Display summary statistics
    print("\nSummary Statistics:")
    print(f"Total shipments across all districts/months: {results['Total shipments'].sum()}")
    print(f"Overall ship percentage: {results['Ship in less than 84 days'].sum() / results['Total shipments'].sum() * 100:.2f}%")
    print(f"Number of unique districts: {results['District Name'].nunique()}")
    print(f"Number of unique months: {results['Referral Month'].nunique()}")
    
    # Save results to Excel (optional)
    results.to_excel('shipment_analysis_results.xlsx', index=False)
    print("\nResults saved to: shipment_analysis_results.xlsx")
else:
    print("\nAnalysis failed. Please check the error messages above for guidance.")